In [1]:
from load_data import data_loader

fa, func, gm = data_loader()

Extracting all the files now...
Done!


In [2]:
import pandas as pd
clinical_data = pd.read_excel("/home/eloydelas/Desktop/CapstoneProject/Capstone_Project/data/subject_clinical_data.xlsx")

In [3]:
#Overview of clincal information data
clinical_data.describe()

,controls_ms,age,sex,dd,mstype,dmd,edss,uGMSSS,oGMSSS,sdmt,sdmtz,pasat,zpasat,zverbalmemory,zvisualmemory,zattention,zfluency,zglobal
count,165.000000,165.000000,165.000000,165.000000,165.000000,146.000000,165.000000,147.000000,147.000000,122.000000,122.000000,136.000000,125.000000,64.000000,64.000000,62.000000,64.000000,62.000000
mean,0.890909,46.161529,0.278788,14.110788,0.060606,2.527397,2.369697,3.031156,2.501483,47.975410,0.079029,44.617647,-0.289255,-0.815819,-0.495052,-0.285719,-0.657595,-0.554303
std,0.312702,10.602227,0.449768,10.051942,0.591577,3.167062,1.802689,2.008814,1.960155,13.351239,1.064729,12.495960,1.333481,1.296972,1.058694,1.038537,1.113514,0.828612
min,0.000000,22.660000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.490000,0.234000,13.000000,-4.166700,8.000000,-5.142900,-3.805600,-3.125000,-3.214300,-3.500000,-3.161200
25%,1.000000,38.210000,0.000000,7.750000,0.000000,0.000000,1.500000,1.415000,0.939500,38.000000,-0.538475,38.750000,-0.857100,-1.689450,-1.400000,-0.868000,-1.500000,-0.975425
50%,1.000000,46.330000,0.000000,12.400000,0.000000,1.000000,2.000000,2.530000,1.922000,49.000000,0.198059,48.500000,0.000000,-0.664200,-0.637500,-0.110450,-0.464300,-0.424400
75%,1.000000,53.840000,1.000000,19.600000,0.000000,4.750000,3.000000,4.395000,3.488000,56.000000,0.638425,54.000000,0.700000,0.090250,0.312500,0.414050,0.148850,-0.057925
max,1.000000,72.220000,1.000000,46.130000,2.000000,14.000000,7.500000,9.290000,9.130000,92.000000,3.000000,60.000000,1.750000,1.910300,1.550000,1.987200,1.833300,0.920800


## FA-weighted correction matrices (age,sex and false positive connections) 

In [4]:
import numpy as np
from sklearn import linear_model
from statistics import mean

# create an empty array
fa_corrected = np.zeros(fa.shape)

X = np.array(list(zip(clinical_data['age'], clinical_data['sex'])))
X.shape

#Correct for age and sex the connectivity matrices
for i in range(fa.shape[1]):
    Y = fa.values[:,i] #Extract the values inside matrix
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)
    y_pred = regr.predict(X)
    residual = (Y - y_pred)
    val_corrected = residual + mean(Y)
    # save values
    fa_corrected[:,i] = val_corrected
    
fa_corrected[fa_corrected < 0.1] = 0


In [5]:
print("Number of negative - zero - positive values are {} - {} - {} / {}".format(len(fa_corrected[fa_corrected < 0]), len(fa_corrected[fa_corrected == 0]), len(fa_corrected[fa_corrected > 0]), fa_corrected.size))

Number of negative - zero - positive values are 0 - 168436 - 301814 / 470250


In [6]:
##Select the index and columns from the original Dataframe
names = fa.index.tolist()
columns = fa.columns.values.tolist()
df=pd.DataFrame(fa_corrected,columns=[columns], index=[names])

In [7]:
# Select HC from the clinic data
num_subjs = clinical_data.shape[0]
ct_control = -1


def get_controls_idx(mstype, ct_control=-1):
  return list(np.where(np.array(mstype)==-1)[0])

controls = get_controls_idx(clinical_data['mstype'], ct_control)

print("Controls indices are [{} of {}]:\n{}".format(len(controls), num_subjs, controls))

#Select the rows belonging to controls cohort
df_HC = df.iloc[controls] # select rows and columns by number, in the order that they appear in the data frame
df_HC.head()

Controls indices are [18 of 165]:
[84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 157, 158, 159, 160, 161, 162, 163, 164]


,ctx-lh-caudalanteriorcingulate/ctx-lh-caudalmiddlefrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-cuneus,ctx-lh-caudalanteriorcingulate/ctx-lh-entorhinal,ctx-lh-caudalanteriorcingulate/ctx-lh-fusiform,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiorparietal,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiortemporal,ctx-lh-caudalanteriorcingulate/ctx-lh-isthmuscingulate,ctx-lh-caudalanteriorcingulate/ctx-lh-lateraloccipital,ctx-lh-caudalanteriorcingulate/ctx-lh-lateralorbitofrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-lingual,...,ctx-rh-superiorparietal/ctx-rh-superiortemporal,ctx-rh-superiorparietal/ctx-rh-supramarginal,ctx-rh-superiorparietal/ctx-rh-transversetemporal,ctx-rh-superiorparietal/ctx-rh-insula,ctx-rh-superiortemporal/ctx-rh-supramarginal,ctx-rh-superiortemporal/ctx-rh-transversetemporal,ctx-rh-superiortemporal/ctx-rh-insula,ctx-rh-supramarginal/ctx-rh-transversetemporal,ctx-rh-supramarginal/ctx-rh-insula,ctx-rh-transversetemporal/ctx-rh-insula
c001MSVIS,0.415356,0.0,0.0,0.0,0.416454,0.460479,0.513861,0.0,0.418106,0.0,...,0.442122,0.378112,0.424537,0.425968,0.379113,0.335230,0.384017,0.382393,0.399733,0.358468
c027MSVIS,0.400464,0.0,0.0,0.0,0.470645,0.458319,0.524846,0.0,0.455811,0.0,...,0.454543,0.420944,0.457751,0.455921,0.361427,0.313259,0.372545,0.380318,0.392884,0.352286
c029MSVIS,0.410177,0.0,0.0,0.0,0.448111,0.000000,0.496964,0.0,0.401597,0.0,...,0.420494,0.384765,0.401099,0.413509,0.359621,0.316739,0.357191,0.365815,0.362908,0.318450
c030MSVIS,0.394410,0.0,0.0,0.0,0.451920,0.000000,0.505629,0.0,0.398886,0.0,...,0.456443,0.404422,0.443293,0.449357,0.371562,0.325713,0.398725,0.373416,0.379563,0.338912
c031MSVIS,0.438183,0.0,0.0,0.0,0.446348,0.460707,0.522138,0.0,0.409560,0.0,...,0.476780,0.420658,0.472600,0.471663,0.376179,0.359368,0.381366,0.380666,0.398167,0.360958


In [31]:
df_HC[df_HC > 0.1] = 1
mask = df_HC.sum(axis = 0, skipna = True)
#bellow 60% values of HC need to be removed the connection
thr = round(df_HC.shape[0]*0.6)
mask[mask < thr] = 0
mask[mask >= thr] = 1
# create an empty array
fa_mask = np.zeros(df.shape)

for num in range(df.shape[0]):
    fa_mask[num,:] = df.values[num,:] * mask # element-wise multiplication

<ipython-input-31-dc118653b9f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_HC[df_HC > 0.1] = 1
/home/eloydelas/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [33]:
#pd.set_option('display.max_rows', None)
sum(mask)

1799.0

In [9]:
##Select the index and columns from the original Dataframe
names = fa.index.tolist()
columns = fa.columns.values.tolist()
fa_corr=pd.DataFrame(fa_mask,columns=[columns], index=[names])

In [10]:
fa.head()

,ctx-lh-caudalanteriorcingulate/ctx-lh-caudalmiddlefrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-cuneus,ctx-lh-caudalanteriorcingulate/ctx-lh-entorhinal,ctx-lh-caudalanteriorcingulate/ctx-lh-fusiform,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiorparietal,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiortemporal,ctx-lh-caudalanteriorcingulate/ctx-lh-isthmuscingulate,ctx-lh-caudalanteriorcingulate/ctx-lh-lateraloccipital,ctx-lh-caudalanteriorcingulate/ctx-lh-lateralorbitofrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-lingual,...,ctx-rh-superiorparietal/ctx-rh-superiortemporal,ctx-rh-superiorparietal/ctx-rh-supramarginal,ctx-rh-superiorparietal/ctx-rh-transversetemporal,ctx-rh-superiorparietal/ctx-rh-insula,ctx-rh-superiortemporal/ctx-rh-supramarginal,ctx-rh-superiortemporal/ctx-rh-transversetemporal,ctx-rh-superiortemporal/ctx-rh-insula,ctx-rh-supramarginal/ctx-rh-transversetemporal,ctx-rh-supramarginal/ctx-rh-insula,ctx-rh-transversetemporal/ctx-rh-insula
002MSVIS,0.395567,0.0,0.0,0.0,0.382439,0.392177,0.490902,0.000000,0.356278,0.0,...,0.383278,0.343662,0.407336,0.405019,0.307111,0.330551,0.378112,0.356339,0.342213,0.345007
003MSVIS,0.330618,0.0,0.0,0.0,0.357768,0.346572,0.457750,0.329869,0.354733,0.0,...,0.379184,0.337306,0.361438,0.381238,0.312535,0.301847,0.359746,0.324277,0.327529,0.341267
004MSVIS,0.418653,0.0,0.0,0.0,0.408343,0.000000,0.492659,0.419281,0.387314,0.0,...,0.407784,0.384566,0.397027,0.405613,0.341303,0.295224,0.356506,0.357396,0.354978,0.349240
005MSVIS,0.378885,0.0,0.0,0.0,0.394806,0.420584,0.478323,0.368988,0.306003,0.0,...,0.436864,0.392562,0.425021,0.420890,0.388629,0.334569,0.398374,0.429484,0.408358,0.378325
010MSVIS,0.354844,0.0,0.0,0.0,0.390623,0.404013,0.452600,0.000000,0.366199,0.0,...,0.395027,0.362122,0.404248,0.408261,0.323118,0.274223,0.337661,0.349933,0.350478,0.344890


In [11]:
df.head()

,ctx-lh-caudalanteriorcingulate/ctx-lh-caudalmiddlefrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-cuneus,ctx-lh-caudalanteriorcingulate/ctx-lh-entorhinal,ctx-lh-caudalanteriorcingulate/ctx-lh-fusiform,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiorparietal,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiortemporal,ctx-lh-caudalanteriorcingulate/ctx-lh-isthmuscingulate,ctx-lh-caudalanteriorcingulate/ctx-lh-lateraloccipital,ctx-lh-caudalanteriorcingulate/ctx-lh-lateralorbitofrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-lingual,...,ctx-rh-superiorparietal/ctx-rh-superiortemporal,ctx-rh-superiorparietal/ctx-rh-supramarginal,ctx-rh-superiorparietal/ctx-rh-transversetemporal,ctx-rh-superiorparietal/ctx-rh-insula,ctx-rh-superiortemporal/ctx-rh-supramarginal,ctx-rh-superiortemporal/ctx-rh-transversetemporal,ctx-rh-superiortemporal/ctx-rh-insula,ctx-rh-supramarginal/ctx-rh-transversetemporal,ctx-rh-supramarginal/ctx-rh-insula,ctx-rh-transversetemporal/ctx-rh-insula
002MSVIS,0.401281,0.0,0.0,0.0,0.375880,0.400833,0.493108,0.000000,0.360572,0.0,...,0.394368,0.354091,0.414272,0.415696,0.313719,0.331984,0.384339,0.359255,0.353211,0.349225
003MSVIS,0.338079,0.0,0.0,0.0,0.345057,0.340937,0.457120,0.317163,0.341322,0.0,...,0.383854,0.342334,0.365249,0.383411,0.318774,0.306799,0.360030,0.332175,0.331107,0.343456
004MSVIS,0.430768,0.0,0.0,0.0,0.391692,0.000000,0.495012,0.385763,0.383830,0.0,...,0.424804,0.400992,0.408261,0.420460,0.353733,0.300302,0.364515,0.366287,0.371159,0.355987
005MSVIS,0.372430,0.0,0.0,0.0,0.404037,0.418244,0.477372,0.385689,0.309504,0.0,...,0.428643,0.384552,0.419488,0.413997,0.382213,0.331597,0.394785,0.424393,0.400668,0.375016
010MSVIS,0.358555,0.0,0.0,0.0,0.383825,0.399265,0.451884,0.000000,0.357345,0.0,...,0.396223,0.363636,0.405541,0.407991,0.325946,0.277040,0.336902,0.354331,0.351019,0.345598


In [12]:
fa_corr.head()

,ctx-lh-caudalanteriorcingulate/ctx-lh-caudalmiddlefrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-cuneus,ctx-lh-caudalanteriorcingulate/ctx-lh-entorhinal,ctx-lh-caudalanteriorcingulate/ctx-lh-fusiform,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiorparietal,ctx-lh-caudalanteriorcingulate/ctx-lh-inferiortemporal,ctx-lh-caudalanteriorcingulate/ctx-lh-isthmuscingulate,ctx-lh-caudalanteriorcingulate/ctx-lh-lateraloccipital,ctx-lh-caudalanteriorcingulate/ctx-lh-lateralorbitofrontal,ctx-lh-caudalanteriorcingulate/ctx-lh-lingual,...,ctx-rh-superiorparietal/ctx-rh-superiortemporal,ctx-rh-superiorparietal/ctx-rh-supramarginal,ctx-rh-superiorparietal/ctx-rh-transversetemporal,ctx-rh-superiorparietal/ctx-rh-insula,ctx-rh-superiortemporal/ctx-rh-supramarginal,ctx-rh-superiortemporal/ctx-rh-transversetemporal,ctx-rh-superiortemporal/ctx-rh-insula,ctx-rh-supramarginal/ctx-rh-transversetemporal,ctx-rh-supramarginal/ctx-rh-insula,ctx-rh-transversetemporal/ctx-rh-insula
002MSVIS,0.401281,0.0,0.0,0.0,0.375880,1.603331,0.493108,0.0,0.360572,0.0,...,0.394368,0.354091,0.414272,0.415696,0.313719,0.331984,0.384339,0.359255,0.353211,0.349225
003MSVIS,0.338079,0.0,0.0,0.0,0.345057,1.363746,0.457120,0.0,0.341322,0.0,...,0.383854,0.342334,0.365249,0.383411,0.318774,0.306799,0.360030,0.332175,0.331107,0.343456
004MSVIS,0.430768,0.0,0.0,0.0,0.391692,0.000000,0.495012,0.0,0.383830,0.0,...,0.424804,0.400992,0.408261,0.420460,0.353733,0.300302,0.364515,0.366287,0.371159,0.355987
005MSVIS,0.372430,0.0,0.0,0.0,0.404037,1.672975,0.477372,0.0,0.309504,0.0,...,0.428643,0.384552,0.419488,0.413997,0.382213,0.331597,0.394785,0.424393,0.400668,0.375016
010MSVIS,0.358555,0.0,0.0,0.0,0.383825,1.597061,0.451884,0.0,0.357345,0.0,...,0.396223,0.363636,0.405541,0.407991,0.325946,0.277040,0.336902,0.354331,0.351019,0.345598


In [ ]:
#import seaborn as sns

#var_corr = fa_corr.corr() #Checking the correlation between each feature
## plot the heatmap and annotation on it
#sns.heatmap(var_corr, xticklabels=var_corr.columns, yticklabels=var_corr.columns, annot=True)